In [ ]:
import pandas as pd
import numpy as np

# import matplotlib as plt
import matplotlib.pyplot as plt
import scipy
from skspatial.objects import LineSegment, Line, Vector
from enum import Enum

Layer = Enum("Layer", "FRONT BACK")

In [ ]:
VIA_DIAM = 0.8
VIA_DRILL = 0.4
STATOR_HOLE_RADIUS = 5
TRACK_WIDTH = 0.127
TRACK_SPACING = 0.127
TURNS = 9
STATOR_RADIUS = 18
PIN_DIAM = 1.7
COIL_CENTER_RADIUS = 11.5
COIL_VIA_RADIUS = 12.5
# where to place the pins
CONNECTION_PINS_RADIUS = 16
USE_SPIRAL = False

In [ ]:
# get the point on an arc at the given angle
def get_arc_point(angle, radius):
    return (
        radius * np.cos(np.deg2rad(angle)),
        radius * np.sin(np.deg2rad(angle)),
    )


# draw an arc
def draw_arc(start_angle, end_angle, radius, step=10):
    points = []
    for angle in np.arange(start_angle, end_angle + step, step):
        x = radius * np.cos(np.deg2rad(angle))
        y = radius * np.sin(np.deg2rad(angle))
        points.append((x, y))
    return points


# roate the points by the required angle
def rotate(points, angle):
    return [
        [
            x * np.cos(np.deg2rad(angle)) - y * np.sin(np.deg2rad(angle)),
            x * np.sin(np.deg2rad(angle)) + y * np.cos(np.deg2rad(angle)),
        ]
        for x, y in points
    ]


# move the points out to the distance at the requited angle
def translate(points, distance, angle):
    return [
        [
            x + distance * np.cos(np.deg2rad(angle)),
            y + distance * np.sin(np.deg2rad(angle)),
        ]
        for x, y in points
    ]


# flip the y coordinate
def flip_y(points):
    return [[x, -y] for x, y in points]


def flip_x(points):
    return [[-x, y] for x, y in points]


def create_pad(radius, angle, name):
    return {
        "x": radius * np.cos(np.deg2rad(angle)),
        "y": radius * np.sin(np.deg2rad(angle)),
        "name": name,
    }


def create_silk(point, text):
    return {
        "x": point[0],
        "y": point[1],
        "text": text,
    }


def create_via(point):
    return {"x": point[0], "y": point[1]}


def create_track(points):
    return [{"x": x, "y": y} for x, y in points]

# Arbitrary Coil Generation

In [ ]:
# templates must be simetric around the X axis and must include the center points on both size (e.g. (X1, 0).... (X2, 0) )
# template must also be convex
template = [
    (-1.5, 0),
    (-1.5, -0.1),
    (1.9, -0.8),
    (1.9, 0.0),
    (1.9, 0.8),
    (-1.5, 0.1),
]

In [ ]:
# plot the template shape wrapping around to the first point
df = pd.DataFrame(template + [template[0]], columns=["x", "y"])
ax = df.plot.line(x="x", y="y", color="blue")
ax.axis("equal")

In [ ]:
def calculate_point(point, point1, point2, spacing, turn):
    reference_vector = Vector([-100, 0])
    angle = np.rad2deg(Vector(point).angle_between(reference_vector))
    if point[1] > 0:
        angle = 360 - angle
    vector = Vector(point1) - Vector(point2)
    normal = vector / np.linalg.norm(vector)
    # rotate the vector 90 degrees
    normal = np.array([-normal[1], normal[0]])
    # move the  point along the normal vector by the spacing
    offset = spacing * (turn * 360 + angle) / 360
    coil_point = point + normal * offset
    return (coil_point[0], coil_point[1])


def get_points(template, turns, spacing):
    coil_points = []
    reference_vector = Vector([-100, 0])
    template_index = 0
    template_length = len(template)
    for turn in range(turns * template_length):
        point1 = template[template_index % template_length]
        point2 = template[(template_index + 1) % template_length]

        # calculate the new positions of the points
        coil_point1 = calculate_point(
            point1, point1, point2, spacing, template_index // template_length
        )
        coil_point2 = calculate_point(
            point2, point1, point2, spacing, (template_index + 1) // template_length
        )
        # adjust the previous point so that the previous line intersects with this new line
        # this prevents any cutting of corners
        if len(coil_points) >= 2:
            # create a line from the previous two points
            line1 = Line(
                coil_points[len(coil_points) - 2],
                np.array(coil_points[len(coil_points) - 1])
                - np.array(coil_points[len(coil_points) - 2]),
            )
            # create a line from the two new points
            line2 = Line(
                np.array(coil_point1),
                np.array(np.array(coil_point1) - np.array(coil_point2)),
            )
            # find the intersection of the two lines
            try:
                intersection = line1.intersect_line(line2)
                # replace the previous point with the intersection
                coil_points[len(coil_points) - 1] = intersection
                # add the new point
                coil_points.append(coil_point2)
            except:
                # the lines did not intersect so just add the points
                coil_points.append(coil_point1)
                coil_points.append(coil_point2)
        else:
            coil_points.append(coil_point1)
            coil_points.append(coil_point2)

        template_index = template_index + 1
    return coil_points


def optimize_points(points):
    # follow the line and remove points that are in the same direction as the previous poin
    # keep doing this until the direction changes significantly
    # this is a very simple optimization that removes a lot of points
    # it's not perfect but it's a good start
    optimized_points = []
    for i in range(len(points)):
        if i == 0:
            optimized_points.append(points[i])
        else:
            vector1 = np.array(points[i]) - np.array(points[i - 1])
            vector2 = np.array(points[(i + 1) % len(points)]) - np.array(points[i])
            length1 = np.linalg.norm(vector1)
            length2 = np.linalg.norm(vector2)
            if length1 > 0 and length2 > 0:
                dot = np.dot(vector1, vector2) / (length1 * length2)
                # clamp dot between -1 and 1
                dot = max(-1, min(1, dot))
                angle = np.arccos(dot)
                if angle > np.deg2rad(5):
                    optimized_points.append(points[i])
    print("Optimised from {} to {} points".format(len(points), len(optimized_points)))
    return optimized_points


def chaikin(points, iterations):
    if iterations == 0:
        return points
    l = len(points)
    smoothed = []
    for i in range(l - 1):
        x1, y1 = points[i]
        x2, y2 = points[i + 1]
        smoothed.append([0.95 * x1 + 0.05 * x2, 0.95 * y1 + 0.05 * y2])
        smoothed.append([0.05 * x1 + 0.95 * x2, 0.05 * y1 + 0.95 * y2])
    smoothed.append(points[l - 1])
    return chaikin(smoothed, iterations - 1)

In [ ]:
if not USE_SPIRAL:
    template_f = []
    for i in range(len(template)):
        template_f.append(template[len(template) - i - len(template) // 2])
    template_f = flip_x(template_f)
    points_f = chaikin(
        optimize_points(
            flip_x(get_points(template_f, TURNS, TRACK_SPACING + TRACK_WIDTH))
        ),
        2,
    )
    points_b = chaikin(
        optimize_points(get_points(template, TURNS, TRACK_SPACING + TRACK_WIDTH)), 2
    )

    points_f = [(COIL_VIA_RADIUS - COIL_CENTER_RADIUS, 0)] + points_f
    points_b = [(COIL_VIA_RADIUS - COIL_CENTER_RADIUS, 0)] + points_b

    df = pd.DataFrame(points_f, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="blue")
    ax.axis("equal")
    df = pd.DataFrame(points_b, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="red", ax=ax)

    print("Track points", len(points_f), len(points_b))
else:
    print("Using spiral")

# Basic Spiral Coil Generation

In [ ]:
def get_spiral(turns, start_radius, thickness, layer=Layer.FRONT):
    points = []
    # create a starting point in the center
    for angle in np.arange(0, turns * 360, 1):
        radius = start_radius + thickness * angle / 360
        if layer == Layer.BACK:
            x = radius * np.cos(np.deg2rad(angle + 180))
            y = radius * np.sin(np.deg2rad(angle + 180))
            points.append((x, -y))
        else:
            x = radius * np.cos(np.deg2rad(angle))
            y = radius * np.sin(np.deg2rad(angle))
            points.append((x, y))
    return points

In [ ]:
if USE_SPIRAL:
    points_f = get_spiral(
        TURNS, VIA_DIAM / 2 + TRACK_SPACING, TRACK_SPACING + TRACK_WIDTH, Layer.FRONT
    )
    points_b = get_spiral(
        TURNS, VIA_DIAM / 2 + TRACK_SPACING, TRACK_SPACING + TRACK_WIDTH, Layer.BACK
    )

    points_f = [(COIL_VIA_RADIUS - COIL_CENTER_RADIUS, 0)] + points_f
    points_b = [(COIL_VIA_RADIUS - COIL_CENTER_RADIUS, 0)] + points_b
    print("Track points", len(points_f), len(points_b))
else:
    print("Using template")

# Generate PCB Layout

In [ ]:
# calculat the total length of the track to compute the resistance
total_length_front = 0
for i in range(len(points_f) - 1):
    total_length_front += np.linalg.norm(
        np.array(points_f[i + 1]) - np.array(points_f[i])
    )
print("Total length front", total_length_front)

total_length_back = 0
for i in range(len(points_b) - 1):
    total_length_back += np.linalg.norm(
        np.array(points_b[i + 1]) - np.array(points_b[i])
    )
print("Total length back", total_length_back)

In [ ]:
vias = []
tracks_f = []
tracks_b = []
pads = []
silk = []

# create the pads at CONNECTION_PINS radius - 2 for each of the coils, A, B and C
# angle_A = 0
# pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_A - 30, "A"))
# pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_A + 30, "A"))

# angle_B = 120
# pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_B - 30, "B"))
# pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_B + 30, "B"))

# angle_C = 240
# pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_C - 30, "C"))
# pads.append(create_pad(CONNECTION_PINS_RADIUS, angle_C + 30, "C"))


# the main coils
coil_labels = ["A", "B", "C"]
coils_f = []
coils_b = []
for i in range(12):
    angle = i * 360 / 12
    if (i // 3) % 2 == 0:
        coil_A_f = translate(rotate(points_f, angle), COIL_CENTER_RADIUS, angle)
        coil_A_b = translate(rotate(points_b, angle), COIL_CENTER_RADIUS, angle)
    else:
        coil_A_f = translate(rotate(flip_y(points_f), angle), COIL_CENTER_RADIUS, angle)
        coil_A_b = translate(rotate(flip_y(points_b), angle), COIL_CENTER_RADIUS, angle)
    # keep track of the coils
    coils_f.append(coil_A_f)
    coils_b.append(coil_A_b)

    tracks_f.append(coil_A_f)
    tracks_b.append(coil_A_b)
    vias.append(create_via(get_arc_point(angle, COIL_VIA_RADIUS)))
    silk.append(
        create_silk(get_arc_point(angle, COIL_CENTER_RADIUS), coil_labels[i % 3])
    )

# raidus for connecting the bottoms of the coils together
connection_radius1 = STATOR_HOLE_RADIUS + TRACK_SPACING + TRACK_WIDTH

# create tracks to link the A coils around the center
connection_via_radius_A = connection_radius1 + TRACK_SPACING + VIA_DIAM / 2
coil_A1_A2_inner = (
    [get_arc_point(0, connection_via_radius_A)]
    + draw_arc(0, 3 * 360 / 12, connection_radius1)
    + [get_arc_point(3 * 360 / 12, connection_via_radius_A)]
)
tracks_f.append(coil_A1_A2_inner)
coil_A3_A4_inner = (
    [get_arc_point(6 * 360 / 12, connection_via_radius_A)]
    + draw_arc(6 * 360 / 12, 9 * 360 / 12, connection_radius1)
    + [get_arc_point(9 * 360 / 12, connection_via_radius_A)]
)
tracks_f.append(coil_A3_A4_inner)
# connect up the bottoms of the A coils
coils_b[0].append(coil_A1_A2_inner[0])
coils_b[3].append(coil_A1_A2_inner[-1])
coils_b[6].append(coil_A3_A4_inner[0])
coils_b[9].append(coil_A3_A4_inner[-1])
# add the vias to stitch them together
vias.append(create_via(coil_A1_A2_inner[0]))
vias.append(create_via(coil_A1_A2_inner[-1]))
vias.append(create_via(coil_A3_A4_inner[0]))
vias.append(create_via(coil_A3_A4_inner[-1]))

# create tracks to link the B coils around the center - this can all be done on the bottom layer
coil_B1_B2_inner = draw_arc(1 * 360 / 12, 4 * 360 / 12, connection_radius1)
tracks_b.append(coil_B1_B2_inner)
coil_B3_B4_inner = draw_arc(7 * 360 / 12, 10 * 360 / 12, connection_radius1)
tracks_b.append(coil_B3_B4_inner)
# connect up the bottoms of the A coils
coils_b[1].append(coil_B1_B2_inner[0])
coils_b[4].append(coil_B1_B2_inner[-1])
coils_b[7].append(coil_B3_B4_inner[0])
coils_b[10].append(coil_B3_B4_inner[-1])

# create tracks to link the C coils around the center
connection_via_radius_C = connection_via_radius_A + TRACK_SPACING + VIA_DIAM / 2
coil_C1_C2_inner = draw_arc(2 * 360 / 12, 5 * 360 / 12, connection_via_radius_C)
tracks_f.append(coil_C1_C2_inner)
coil_C3_C4_inner = draw_arc(8 * 360 / 12, 11 * 360 / 12, connection_via_radius_C)
tracks_f.append(coil_C3_C4_inner)
# connect up the bottoms of the B coils
coils_b[2].append(coil_C1_C2_inner[0])
coils_b[5].append(coil_C1_C2_inner[-1])
coils_b[8].append(coil_C3_C4_inner[0])
coils_b[11].append(coil_C3_C4_inner[-1])
# add the vias to stitch them together
vias.append(create_via(coil_C1_C2_inner[0]))
vias.append(create_via(coil_C1_C2_inner[-1]))
vias.append(create_via(coil_C3_C4_inner[0]))
vias.append(create_via(coil_C3_C4_inner[-1]))

# connect the last three coils together
common_connection_radius = STATOR_RADIUS - TRACK_SPACING - TRACK_WIDTH
tracks_f.append(draw_arc(9 * 360 / 12, 11 * 360 / 12, common_connection_radius))
coils_f[9].append(get_arc_point(9 * 360 / 12, common_connection_radius))
coils_f[10].append(get_arc_point(10 * 360 / 12, common_connection_radius))
coils_f[11].append(get_arc_point(11 * 360 / 12, common_connection_radius))

# connect the outer A coils together
outer_connection_radius_A = STATOR_RADIUS - TRACK_SPACING - TRACK_WIDTH
tracks_f.append(draw_arc(3 * 360 / 12, 6 * 360 / 12, outer_connection_radius_A))
coils_f[3].append(get_arc_point(3 * 360 / 12, outer_connection_radius_A))
coils_f[6].append(get_arc_point(6 * 360 / 12, outer_connection_radius_A))

# connect the outer B coils together
outer_connection_radius_B = outer_connection_radius_A - TRACK_SPACING - VIA_DIAM / 2
tracks_b.append(
    [get_arc_point(4 * 360 / 12, outer_connection_radius_B)]
    + draw_arc(4 * 360 / 12, 7 * 360 / 12, outer_connection_radius_A)
    + [get_arc_point(7 * 360 / 12, outer_connection_radius_B)]
)
coils_f[4].append(get_arc_point(4 * 360 / 12, outer_connection_radius_B))
coils_f[7].append(get_arc_point(7 * 360 / 12, outer_connection_radius_B))
vias.append(create_via(get_arc_point(4 * 360 / 12, outer_connection_radius_B)))
vias.append(create_via(get_arc_point(7 * 360 / 12, outer_connection_radius_B)))

# connect the outer C coilds together
outer_connection_radius_C = outer_connection_radius_B - TRACK_SPACING - VIA_DIAM / 2
tracks_b.append(draw_arc(5 * 360 / 12, 8 * 360 / 12, outer_connection_radius_C))
coils_f[5].append(get_arc_point(5 * 360 / 12, outer_connection_radius_C))
coils_f[8].append(get_arc_point(8 * 360 / 12, outer_connection_radius_C))
vias.append(create_via(get_arc_point(5 * 360 / 12, outer_connection_radius_C)))
vias.append(create_via(get_arc_point(8 * 360 / 12, outer_connection_radius_C)))

# create pads for the input
pin_radius = STATOR_RADIUS - TRACK_SPACING - PIN_DIAM / 2
pads.append(create_pad(pin_radius, 0, "A"))
pads.append(create_pad(pin_radius, 1 * 360 / 12, "B"))
pads.append(create_pad(pin_radius, 2 * 360 / 12, "C"))

coils_f[0].append(get_arc_point(0, pin_radius))
coils_f[1].append(get_arc_point(1 * 360 / 12, pin_radius))
coils_f[2].append(get_arc_point(2 * 360 / 12, pin_radius))

In [ ]:
# dump out the results to json
json_result = {
    "parameters": {
        "trackWidth": TRACK_WIDTH,
        "statorHoleRadius": STATOR_HOLE_RADIUS,
        "statorRadius": STATOR_RADIUS,
        "viaDiameter": VIA_DIAM,
        "viaDrillDiameter": VIA_DRILL,
    },
    "vias": vias,
    "pads": pads,
    "silk": silk,
    "tracks": {
        "f": [create_track(points) for points in tracks_f],
        "b": [create_track(points) for points in tracks_b],
    },
}

import json

json.dump(json_result, open("coil.json", "w"))


# df = pd.DataFrame(coil_A_f, columns=["x", "y"])
# ax = df.plot.line(x="x", y="y", label="Coil A", color="blue")
# ax.axis("equal")
# df = pd.DataFrame(coil_A_b, columns=["x", "y"])
# ax = df.plot.line(x="x", y="y", label="Coil B", color="green")
# ax.axis("equal")


# plot the back tracks
ax = None
for track in json_result["tracks"]["b"]:
    df = pd.DataFrame(track, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="blue", ax=ax)
    ax.axis("equal")

# plot the front tracks
for track in json_result["tracks"]["f"]:
    df = pd.DataFrame(track, columns=["x", "y"])
    ax = df.plot.line(x="x", y="y", color="red", ax=ax)
    ax.axis("equal")

# hide the legend
ax.legend().set_visible(False)
# make the plot bigger
ax.figure.set_size_inches(10, 10)

# plot the vias
for via in json_result["vias"]:
    ax.add_patch(
        plt.Circle(
            (via["x"], via["y"]),
            radius=VIA_DIAM / 2,
            fill=True,
            color="black",
        )
    )
    ax.add_patch(
        plt.Circle(
            (via["x"], via["y"]),
            radius=VIA_DRILL / 2,
            fill=True,
            color="white",
        )
    )

# plot the edge cuts
ax.add_patch(
    plt.Circle(
        (0, 0),
        radius=STATOR_RADIUS,
        fill=False,
        color="yellow",
    )
)
ax.add_patch(
    plt.Circle(
        (0, 0),
        radius=STATOR_HOLE_RADIUS,
        fill=False,
        color="yellow",
    )
)

# plot the pads
for pad in json_result["pads"]:
    ax.add_patch(
        plt.Circle(
            (pad["x"], pad["y"]),
            radius=1.7 / 2,
            fill=True,
            color="yellow",
        )
    )
    ax.add_patch(
        plt.Circle(
            (pad["x"], pad["y"]),
            radius=1.0 / 2,
            fill=True,
            color="white",
        )
    )

# plot the silk
for silk in json_result["silk"]:
    ax.text(
        silk["x"],
        silk["y"],
        silk["text"],
        horizontalalignment="center",
        verticalalignment="center",
        color="black",
        fontsize=50,
    )